In [ ]:
import os
import pandas as pd
import sqlite3

import download_params

In [ ]:
def get_all_csvs(path):
    csv_files = []
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.split(".")[-1] == "csv":
                csv_files.append(os.path.join(root, file))

    return csv_files

In [ ]:
def get_col_types_for_db(df, primary_key=None):
    col_types = []
    for col in df.columns:
        data_type = ""
        if df[col].dtype == int:
            data_type = "INTEGER"
        elif df[col].dtype == float:
            data_type = "REAL"
        elif df[col].dtype == object:
            data_type = "TEXT"
        if col == primary_key:
            data_type = data_type + " PRIMARY KEY"
        col_types.append(data_type)
    return col_types




In [ ]:
def make_table_command(table_name, columns, col_types):
# make airport table
    command = f"CREATE TABLE {table_name} ("
    for i, col in enumerate(columns):
        command = command + f"{col} {col_types[i]}, "
    return command[:-2] + ")"

In [ ]:
def make_database(database_path, df):
    cols_id = [
        "FlightDate",
        "Reporting_Airline", 
        "OriginAirportID", 
        "DestAirportID",
    ]
    cols_time = [
        "Year", 
        "Quarter", 
        "Month", 
        "DayofMonth", 
        "DayOfWeek", 
        "FlightDate",
    ]
    cols_airline = [
        "Reporting_Airline",
        "DOT_ID_Reporting_Airline",
        "IATA_CODE_Reporting_Airline",
        "Tail_Number",
        "Flight_Number_Reporting_Airline",
    ]
    cols_airport_origin = [
        "OriginAirportID",
        "OriginAirportSeqID",
        "OriginCityMarketID",
        "Origin",
        "OriginCityName",
        "OriginState",
        "OriginStateFips",
        "OriginStateName",
        "OriginWac",
    ]
    cols_airport_dest = [
        "DestAirportID",
        "DestAirportSeqID",
        "DestCityMarketID",
        "Dest",
        "DestCityName",
        "DestState",
        "DestStateFips",
        "DestStateName",
        "DestWac",
    ]
    cols_flight = [
        col for col in df.columns \
        if (col not in cols_time \
        and col not in cols_airline \
        and col not in cols_airport_origin \
        and col not in cols_airport_dest) \
        or col in cols_id
    ]
    cols_airport = [
        col[4:] if col != "Dest" else "Airport" for col in cols_airport_dest
    ]

    make_table_commands = []
    make_table_commands.append(
        make_table_command(
            "time_period", cols_time, get_col_types_for_db(df[cols_time], primary_key="FlightDate",)
        )
    )
    make_table_commands.append(
        make_table_command(
            "airports", cols_airport, get_col_types_for_db(df[cols_airport_dest], primary_key="DestAirportID")
        )
    )
    make_table_commands.append(
        make_table_command(
            "airlines", cols_airline, get_col_types_for_db(df[cols_airline], primary_key="Reporting_Airline",)
        )
    )
    make_table_commands.append(
        make_table_command(
            "flights", cols_flight, get_col_types_for_db(df[cols_flight])
        )
    )

    conn = sqlite3.connect(database_path)
    crsr = conn.cursor()

    for cmd in make_table_commands:
        try:
            crsr.execute(cmd)
        except Exception as e:
            print(e)
    crsr.close()
    conn.close()

    return {"time_period": cols_time, "airports": cols_airport, "airlines": cols_airline, "flights": cols_flight}



In [ ]:
def get_uniques(df, column_uniques):
    # uniques = df[column_uniques].unique
    # column_uniques = "DayOfWeek"
    uniques = df[column_uniques].unique()
    df_uniques = pd.DataFrame()
    for val in uniques:
        df_uniques = pd.concat([df_uniques, df[df[column_uniques]==val].iloc[:1,:]])
    return df_uniques


In [ ]:
db_name = "on_time_performance.sqlitedb"
db_path = ""
db_path = os.path.join(db_path, db_name)
csv_files = get_all_csvs(download_params.DOWNLOAD_PATH)
df_time_periods = pd.DataFrame()
df_airports = pd.DataFrame()
df_airlines = pd.DataFrame()

# months = 0

for i, csv_file in enumerate(csv_files):
    df = pd.read_csv(csv_file, low_memory=False)
    for col in df.columns:
        if "Unnamed" in col:
            df.drop(col, axis=1, inplace=True)
    if not i:
        db_tables = make_database(db_path, df)
        conn = sqlite3.connect(db_path)
    # add_data_to_db("on_time_performance", db_path, df, col_groups)
    df[db_tables["flights"]].to_sql("flights", conn, if_exists="append", index=False)
    
    # airports
    dest_airports = df[[f"Dest{col}" if col != "Airport" else "Dest" for col in db_tables["airports"]]]
    origin_airports = df[[f"Origin{col}" if col != "Airport" else "Origin" for col in db_tables["airports"]]]
    dest_airports.columns = origin_airports.columns = db_tables["airports"]
    airports = pd.concat([dest_airports, origin_airports])
    airports = get_uniques(airports, "AirportID")
    df_airports = pd.concat([df_airports, airports])

    # airlines
    airlines = get_uniques(df[db_tables["airlines"]], "Reporting_Airline")
    df_airlines = pd.concat([df_airlines, airlines])

    # time periods
    time_periods = get_uniques(df[db_tables["time_period"]], "FlightDate")
    df_time_periods = pd.concat([df_time_periods, time_periods])

df_time_periods.to_sql("time_period", conn, if_exists="append", index=False)
get_uniques(df_airports, "AirportID").to_sql("airports", conn, if_exists="append", index=False)
get_uniques(df_airlines, "Reporting_Airline").to_sql("airlines", conn, if_exists="append", index=False)

conn.close()
